# 对抗生成网络与神经网络风格迁移的结合



## 概述

大家经过这几天对 NST（神经网络风格迁移）和DCGAN（深度卷积神经网络）的学习，已经对这两个经典的机器学习模型有了一定的认知。接下来我们就讲这两个技术进行结合。

## 环境搭建


###  所需包的导入

In [ ]:
from __future__ import print_function, division
import keras
from keras.preprocessing.image import load_img, save_img, img_to_array
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

import sys,os
import numpy as np
import sys

###  数据下载与准备

In [ ]:
if not os.path.exists('NST/data'):
    os.makedirs('NST/data') 

if not os.path.exists('GANS/models'):
    os.makedirs('GANS/models') 

# !wget -q https://github.com/JChrysanthemum/deepflying/raw/master/InnoCamp/NST/pics/content.png -O NST/data/c1.png
# !wget -q https://github.com/JChrysanthemum/deepflying/raw/master/InnoCamp/NST/pics/style.jpg -O NST/data/s1.jpg
# !wget -q https://github.com/JChrysanthemum/deepflying/raw/master/InnoCamp/GANS/models/models.zip -O GAN/models/models.zip
# !wget -q https://github.com/JChrysanthemum/deepflying/raw/master/DraftVersion/NST/styleTransfer.py -O DeepFlying/styleTransfer.py
# !unzip -o GAN/models/models.zip -d GAN/models/

from NST import styleTransfer as st


## 代码编写

### 参数定义

In [ ]:
st.target_image_path = "gan.png"
st.style_reference_path = "NST/data/s1.jpg"

#@markdown 迭代次数
st.iter_size = 10 #@param {type:"slider", min:10, max:200, step:5}
#@markdown 生成器模型
generator_path = 'shirt' #@param ['ankle_boot', 'bag', 'coat', 'dress','pullover','sandal','shirt','sneaker','trouser']
generator_path = 'GANS/models/'+ generator_path + '.h5'

# 显卡编号 可选 0 或 1
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf 
from keras.backend.tensorflow_backend import set_session 

# 显卡使用率 方便大家共同使用请勿更改此数值
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2) 
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


### 构造生成器

In [ ]:
class Generator:
    def __init__(self):
        self.generator = keras.models.load_model(generator_path)
        self.generator.trainable =False
        

    def save_imgs(self):

        noise = np.random.normal(0, 1, (1, 100))
        gen_imgs = self.generator.predict(noise)
        
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        # Write these images to files
        fig, axs = plt.subplots(1,1)
        plt.imshow(gen_imgs[ 0, :, :, 0], cmap='gray')
        gen_imgs=cv2.cvtColor(gen_imgs[ 0, :, :, 0],cv2.COLOR_GRAY2BGR)*255
        
        cv2.imwrite("gan.png",gen_imgs)

        return gen_imgs
      
generator = Generator()
img = generator.save_imgs()



### 进行风格迁移

In [ ]:

print(st.main())

gan = load_img("gan.png")
gan = img_to_array(gan)
gan = np.array(gan, np.uint8)

nst = load_img("NST.png")
nst = img_to_array(nst)
nst = np.array(nst, np.uint8)

plt.figure(figsize=(5, 6))
plt.subplot(2, 3, 1), plt.title('GAN')
plt.axis('off')
plt.imshow(gan)
plt.subplot(2, 3, 2), plt.title('NST')
plt.axis('off')
plt.imshow(nst)
plt.show()